In [1]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

from ipywidgets import StaticInteract, RadioWidget, RangeWidget
from scipy.interpolate import interp1d

In [2]:
rc('font',**{'size': 20})
rc('text', usetex=True)

In [3]:
def create_parameters_and_stables_files():

    os.system('mkdir results/')

    masses_directories = os.listdir('./particles/')

    with open('./results/parameters.txt', 'w') as outfile:
        outfile.write('m_ffp\tv_ffp\tb_ffp\tphi_bp\tinc_bp\tlan_bp\tap_bp\te_star_ffp\te_star_bp\tsma_star_ffp\tsma_star_bp\tinc_star_ffp\tinc_star_bp\tlan_star_ffp\tlan_star_bp\tap_star_ffp\tap_star_bp\trun_time\tenergy_change\tintegration_time\n')
        for i in range(len(masses_directories)):
            mass_dir = masses_directories[i]
            fname = './particles/'+mass_dir+'/parameters_'+mass_dir+'.txt'
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
            infile.close()
    outfile.close()

    with open('./results/stables.txt', 'w') as outfile:
        outfile.write('m_ffp\tv_ffp\tb_ffp\tphi_bp\tinc_bp\tlan_bp\tap_bp\te_star_ffp\te_star_bp\tsma_star_ffp\tsma_star_bp\tinc_star_ffp\tinc_star_bp\tlan_star_ffp\tlan_star_bp\tap_star_ffp\tap_star_bp\trun_time\tenergy_change\tintegration_time\n')
        for i in range(len(masses_directories)):
            mass_dir = masses_directories[i]
            fname = './particles/'+mass_dir+'/stables_'+mass_dir+'.txt'
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
            infile.close()
    outfile.close()

def read_parameters_df():

    df = pd.read_csv('./results/parameters.txt', sep='\t', dtype=np.float64)

    ms_ffp = df['m_ffp']
    vs_ffp = df['v_ffp']
    bs_ffp = df['b_ffp']

    return df, ms_ffp, vs_ffp, bs_ffp

def read_stables_df():

    df = pd.read_csv('./results/stables.txt', sep='\t', dtype=np.float64)

    ms_ffp = df['m_ffp']
    vs_ffp = df['v_ffp']
    bs_ffp = df['b_ffp']

    return df, ms_ffp, vs_ffp, bs_ffp
    
def create_plots_folders():
    os.system('mkdir plots/')

In [4]:
def calculate_cross_section(b_ffp_max, number_captures, number_total):
    
    cs = np.pi*b_ffp_max**2*number_captures/number_total
    
    return cs

In [5]:
#Parameters and Stables Files
create_parameters_and_stables_files()

#Read df
df, ms_ffp, vs_ffp, bs_ffp = read_stables_df()
df_par, ms_ffp_par, vs_ffp_par, bs_ffp_par = read_parameters_df()

ms_ffp_par_uniq = np.unique(ms_ffp_par)
vs_ffp_par_uniq = np.unique(vs_ffp_par)

print 'Stables: ', len(ms_ffp), '/', len(ms_ffp_par), '->', len(ms_ffp)*100/len(ms_ffp_par), '%'

#Plots
create_plots_folders()

Stables:  78851 / 600000 -> 13 %


In [6]:
def plot_cross_sections_vs_velocity(m_ffp_ind):
    
    m_ffp = ms_ffp_par_uniq[m_ffp_ind]
        
    vs_ffp_captured = np.array(vs_ffp)[np.where(ms_ffp == m_ffp)]
    bs_ffp_captured = np.array(bs_ffp)[np.where(ms_ffp == m_ffp)]

    vs_ffp_total = np.array(vs_ffp_par)[np.where(ms_ffp_par == m_ffp)]
    bs_ffp_total = np.array(bs_ffp_par)[np.where(ms_ffp_par == m_ffp)]
    
    x_axis = []
    y_axis = []
    y_axis_errors_up = []
    y_axis_errors_low = []
    
    unique_velocities, number_unique_velocities = np.unique(vs_ffp_total, return_counts=True)
    
    for uniq_vel in unique_velocities:
        
        indices_captured = np.where(vs_ffp_captured == uniq_vel)
        indices_total = np.where(vs_ffp_total == uniq_vel)
        bs_ffp_captured_for_v = bs_ffp_captured[indices_captured]
        
        if (len(bs_ffp_captured_for_v)==0):
            b_ffp_captured_for_v_max = 0.0
        else:
            b_ffp_captured_for_v_max = max(bs_ffp_captured_for_v)
        
        number_captures = len(indices_captured[0])      
        number_total = len(indices_total[0])
        cs = calculate_cross_section(b_ffp_captured_for_v_max, number_captures, number_total)
        
        if(cs != 0.0):
            log_cs = np.log10(cs)
            
            error_up = cs+cs/max(1.0,np.sqrt(number_captures))
            error_low = cs-cs/max(1.0,np.sqrt(number_captures))
            
            log_cs_error_up = np.log10(error_up)
            
            if(error_low != 0.0):
                log_cs_error_low = np.log10(error_low)
            else:
                log_cs_error_low = 0.0
            
            x_axis.append(uniq_vel)
            y_axis.append(log_cs)
            y_axis_errors_up.append(log_cs_error_up)
            y_axis_errors_low.append(log_cs_error_low)
            
    figure = plt.figure(figsize=(15,8))    
    fig = figure.add_subplot(1,1,1)
    
    m_ffp_string = r" ${0:.3f}$".format(m_ffp)
    
    fig.plot(x_axis, y_axis, c='m', marker='o', label='$m_{FFP}= $'+m_ffp_string+'$\mathrm{\quad (MJupiter)}$')
    fig.errorbar(x_axis, y_axis, xerr=None, yerr=[y_axis_errors_low, y_axis_errors_up], ecolor='black')
    fig.axvline(unique_velocities[0]*10.0/8.0, -10, 10, c='black', linestyle='--', alpha=0.6)
    
    fig.set_xlim(2.2,4.0)
    fig.set_ylim(-2.0, 10)
    
    fig.legend(fontsize=20)    
    fig.set_xlabel('$v_{FFP}\quad \mathrm{(km/s)}$', fontsize=30)
    fig.set_ylabel('$\log(\sigma_{capt}/\mathrm{AU}^2)$', fontsize=30) 
    
    plt.close(figure)
    
    return figure

In [7]:
StaticInteract(plot_cross_sections_vs_velocity, m_ffp_ind=RangeWidget(0,len(ms_ffp_par_uniq)-1,1))

In [8]:
def plot_cross_sections_vs_velocity_no_errbar(m_ffp_ind):
    
    m_ffp = ms_ffp_par_uniq[m_ffp_ind]
        
    vs_ffp_captured = np.array(vs_ffp)[np.where(ms_ffp == m_ffp)]
    bs_ffp_captured = np.array(bs_ffp)[np.where(ms_ffp == m_ffp)]

    vs_ffp_total = np.array(vs_ffp_par)[np.where(ms_ffp_par == m_ffp)]
    bs_ffp_total = np.array(bs_ffp_par)[np.where(ms_ffp_par == m_ffp)]
    
    x_axis = []
    y_axis = []
    
    unique_velocities, number_unique_velocities = np.unique(vs_ffp_total, return_counts=True)
    
    for uniq_vel in unique_velocities:
        
        indices_captured = np.where(vs_ffp_captured == uniq_vel)
        indices_total = np.where(vs_ffp_total == uniq_vel)
        bs_ffp_captured_for_v = bs_ffp_captured[indices_captured]
        
        if (len(bs_ffp_captured_for_v)==0):
            b_ffp_captured_for_v_max = 0.0
        else:
            b_ffp_captured_for_v_max = max(bs_ffp_captured_for_v)
        
        number_captures = len(indices_captured[0])      
        number_total = len(indices_total[0])
        cs = calculate_cross_section(b_ffp_captured_for_v_max, number_captures, number_total)
        
        if(cs != 0.0):
            log_cs = np.log10(cs)
            x_axis.append(uniq_vel)
            y_axis.append(log_cs)
            
    figure = plt.figure(figsize=(15,8))    
    fig = figure.add_subplot(1,1,1)
    
    m_ffp_string = r" ${0:.3f}$".format(m_ffp)
    
    fig.plot(x_axis, y_axis, c='m', marker='o', label='$m_{FFP}= $'+m_ffp_string+'$\mathrm{\quad (MJupiter)}$')
    fig.axvline(unique_velocities[0]*10.0/8.0, -10, 10, c='black', linestyle='--', alpha=0.6)
    
    fig.set_xlim(2.3,4.0)
    fig.set_ylim(-1.3, 5.1)
    
    fig.legend(fontsize=20)    
    fig.set_xlabel('$v_{FFP}\quad \mathrm{(km/s)}$', fontsize=30)
    fig.set_ylabel('$\log(\sigma_{capt}/\mathrm{AU}^2)$', fontsize=30) 
    
    plt.close(figure)
    
    return figure

In [9]:
StaticInteract(plot_cross_sections_vs_velocity_no_errbar, m_ffp_ind=RangeWidget(0,len(ms_ffp_par_uniq)-1,1))

In [10]:
def plot_cross_sections_vs_velocity_soften(m_ffp_ind):
    
    m_ffp = ms_ffp_par_uniq[m_ffp_ind]
        
    vs_ffp_captured = np.array(vs_ffp)[np.where(ms_ffp == m_ffp)]
    bs_ffp_captured = np.array(bs_ffp)[np.where(ms_ffp == m_ffp)]

    vs_ffp_total = np.array(vs_ffp_par)[np.where(ms_ffp_par == m_ffp)]
    bs_ffp_total = np.array(bs_ffp_par)[np.where(ms_ffp_par == m_ffp)]
    
    x_axis = []
    y_axis = []
    
    unique_velocities, number_unique_velocities = np.unique(vs_ffp_total, return_counts=True)
    
    for uniq_vel in unique_velocities:
        
        indices_captured = np.where(vs_ffp_captured == uniq_vel)
        indices_total = np.where(vs_ffp_total == uniq_vel)
        bs_ffp_captured_for_v = bs_ffp_captured[indices_captured]
        
        if (len(bs_ffp_captured_for_v)==0):
            b_ffp_captured_for_v_max = 0.0
        else:
            b_ffp_captured_for_v_max = max(bs_ffp_captured_for_v)
        
        number_captures = len(indices_captured[0])      
        number_total = len(indices_total[0])
        cs = calculate_cross_section(b_ffp_captured_for_v_max, number_captures, number_total)
        
        if(cs != 0.0):
            log_cs = np.log10(cs)
            x_axis.append(uniq_vel)
            y_axis.append(log_cs)
            
    figure = plt.figure(figsize=(15,8))    
    fig = figure.add_subplot(1,1,1)
    
    m_ffp_string = r" ${0:.3f}$".format(m_ffp)
    
    if(len(x_axis) != 0):
    
        softener = interp1d(x_axis, y_axis, kind='linear')
        new_x_axis = np.linspace(x_axis[0], x_axis[-1], 1000)
        new_y_axis = softener(new_x_axis)

        fig.plot(new_x_axis, new_y_axis, c='m', lw=2, label='$m_{FFP}= $'+m_ffp_string+'$\mathrm{\quad (MJupiter)}$')
    
    fig.axvline(unique_velocities[0]*10.0/8.0, -10, 10, c='black', linestyle='--', alpha=0.6)
    
    fig.set_xlim(2.3,4.0)
    fig.set_ylim(-1.3, 5.1)
    
    fig.legend(fontsize=20)    
    fig.set_xlabel('$v_{FFP}\quad \mathrm{(km/s)}$', fontsize=30)
    fig.set_ylabel('$\log(\sigma_{capt}/\mathrm{AU}^2)$', fontsize=30) 
    
    plt.close(figure)
    
    return figure

In [11]:
StaticInteract(plot_cross_sections_vs_velocity_soften, m_ffp_ind=RangeWidget(0,len(ms_ffp_par_uniq)-1,1))

/usr/lib/python2.7/dist-packages/matplotlib/axes/_axes.py:476: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [12]:
def plot_cross_sections_vs_mass_no_errbar(v_ffp_ind):
    
    v_ffp = vs_ffp_par_uniq[v_ffp_ind]
        
    ms_ffp_captured = np.array(ms_ffp)[np.where(vs_ffp == v_ffp)]
    bs_ffp_captured = np.array(bs_ffp)[np.where(vs_ffp == v_ffp)]

    ms_ffp_total = np.array(ms_ffp_par)[np.where(vs_ffp_par == v_ffp)]
    bs_ffp_total = np.array(bs_ffp_par)[np.where(vs_ffp_par == v_ffp)]
    
    x_axis = []
    y_axis = []
    
    unique_masses, number_unique_masses = np.unique(ms_ffp_total, return_counts=True)
    
    for uniq_mass in unique_masses:
        
        indices_captured = np.where(ms_ffp_captured == uniq_mass)
        indices_total = np.where(ms_ffp_total == uniq_mass)
        bs_ffp_captured_for_m = bs_ffp_captured[indices_captured]
        
        if (len(bs_ffp_captured_for_m)==0):
            b_ffp_captured_for_m_max = 0.0
        else:
            b_ffp_captured_for_m_max = max(bs_ffp_captured_for_m)
        
        number_captures = len(indices_captured[0])      
        number_total = len(indices_total[0])
        cs = calculate_cross_section(b_ffp_captured_for_m_max, number_captures, number_total)
        
        if(cs != 0.0):
            log_cs = np.log10(cs)
            x_axis.append(uniq_mass)
            y_axis.append(log_cs)
            
    figure = plt.figure(figsize=(15,8))    
    fig = figure.add_subplot(1,1,1)
    
    v_ffp_string = r" ${0:.3f}$".format(v_ffp)
    
    fig.plot(x_axis, y_axis, c='m', marker='o', label='$v_{FFP}= $'+v_ffp_string+'$\mathrm{\quad (km/s)}$')
    
    fig.set_xlim(0.0,10.0)
    fig.set_ylim(-1.5, 5.0)
    
    fig.legend(fontsize=20)    
    fig.set_xlabel('$m_{FFP}\quad \mathrm{(MJupiter)}$', fontsize=30)
    fig.set_ylabel('$\log(\sigma_{capt}/\mathrm{AU}^2)$', fontsize=30) 
    
    plt.close(figure)
    
    return figure

In [13]:
StaticInteract(plot_cross_sections_vs_mass_no_errbar, v_ffp_ind=RangeWidget(0,len(vs_ffp_par_uniq)-1,1))